In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate

In [ ]:
from datasets import Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from numpy import argmax
from sklearn.metrics import mean_squared_error
from math import sqrt
from evaluate import load
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
from google.colab import drive

Preprocessed ve BERT dataseti, ikisi de özetlenmiş halde

In [ ]:
drive.mount('/content/drive')
root = "/content/drive/My Drive/Ara Dönem Proje Fake News Detection/Verisetleri"
#dataset name
file_name = "dataset_sum_bert.csv"
df = pd.read_csv(f"{root}/{file_name}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

,text,class
0,galatasaray'ın odeabank'ta top koşturduğu orta...,True
1,i̇mparator akihito'nun konuk ağırlama odası ja...,FAKE
2,anneler günü zaman kutlanacak merak edenler gü...,True
3,konya'da bir pazar yerinde hesap yüzünden çıka...,True
4,türkiye'den i̇zmir atatürk eğitim araştırma ha...,True
...,...,...
25152,fenerbahçe'de konyaspor maçı öncesi lens'in yo...,True
25153,adana'da eşiyle bindiği asansörün altında kala...,True
25154,trabzon'da eğlence sırasında havaya ateş eden ...,True
25155,avrupa olimpiyat şampiyonu milli tekvandocu se...,True


In [ ]:
df.rename(columns = {'label':'class'}, inplace = True)

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
#Label Encoder 
label_to_id = {"True":1,"FAKE":0}
id_to_label = {0:"FAKE", 1:"True"}
num_labels = len(label_to_id)
label_to_id

{'True': 1, 'FAKE': 0}

In [ ]:
#Test-Train Split
dataset = dataset.train_test_split(test_size = 0.3, seed = 42)

In [ ]:
#Turkce dataset icin tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], max_length = 128, padding = "max_length", truncation = True)
    return {"input_ids": tokenized.input_ids, "token_type_ids": tokenized.token_type_ids,
            "attention_mask": tokenized.attention_mask, "labels": [label_to_id[label] for label in examples["class"]]}

In [ ]:
#tokanizer olmus dataset
tokenized_datasets = dataset.map(tokenize_function, batched = True)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

In [ ]:
#Turkce dataset icin model 
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", classifier_dropout = 0.1, num_labels = num_labels)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [ ]:
#model output path
path_output = '/content/drive/My Drive/Ara Dönem Proje Fake News Detection/output'
training_args = TrainingArguments(output_dir = path_output, evaluation_strategy = "epoch", per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32, weight_decay = 0.01, num_train_epochs = 4, save_strategy = "epoch")

In [ ]:
metric = load('accuracy')

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = argmax(logits, axis = -1)
    out_file.write(classification_report(labels, predictions) + "\n\n\n")
    confusion_matrices.append(confusion_matrix(labels, predictions, labels = list(id_to_label.keys())))
    return metric.compute(predictions = predictions, references = labels)

In [ ]:
trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = tokenized_datasets["train"],
                  eval_dataset = tokenized_datasets["test"],
                  compute_metrics = compute_metrics)

In [ ]:
#classification reportlari yazdigi dosya
with open(f"{root}/result_sum_bert.txt", 'w') as out_file:
    confusion_matrices = []
    trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, class. If text, class are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17609
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2204
  Number of trainable parameters = 110618882


Epoch,Training Loss,Validation Loss,Accuracy
1,0.056000,0.028558,0.992183
2,0.017800,0.028271,0.993773
3,0.005100,0.035335,0.994038
4,0.000700,0.037558,0.994303


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, class. If text, class are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7548
  Batch size = 32
Saving model checkpoint to /content/drive/My Drive/Ara Dönem Proje Fake News Detection/output/checkpoint-551
Configuration saved in /content/drive/My Drive/Ara Dönem Proje Fake News Detection/output/checkpoint-551/config.json
Model weights saved in /content/drive/My Drive/Ara Dönem Proje Fake News Detection/output/checkpoint-551/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, class. If text, class are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  

In [ ]:
for cf in confusion_matrices:
    print(cf)

[[1399    0]
 [   1 6013]]
[[1399    0]
 [   1 6013]]
[[1399    0]
 [   1 6013]]
[[1399    0]
 [   1 6013]]


In [ ]:
#save model
trainer.save_model(output_dir='/content/drive/My Drive/Ara Dönem Proje Fake News Detection/')

Saving model checkpoint to /content/drive/My Drive/Ara Dönem Proje Fake News Detection/
Configuration saved in /content/drive/My Drive/Ara Dönem Proje Fake News Detection/config.json
Model weights saved in /content/drive/My Drive/Ara Dönem Proje Fake News Detection/pytorch_model.bin
